In [181]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

# Reading data
data = pd.read_csv("merged_data.csv")

In [ ]:
# *********************************************************************************************************************
# Predict delay bin including 0 delay
# *********************************************************************************************************************

# Loading Inputs
input_features = ["wind_dir", "wind_speed", "ceiling", "visibility", "temp", "dew_pnt", "pressure", "congestion_score", "time_of_day", "time_diff"]
X = data[input_features]

# Loading Outputs (5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, inf)
y = data["delay_bin"]

# Splitting data into training and testing sets
X_train, X_test = X[:int(len(data)*0.75)], X[int(len(data)*0.75):]
y_train, y_test = y[:int(len(data)*0.75)], y[int(len(data)*0.75):]

# Building the model
model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,              # fraction of samples per tree
    colsample_bytree=0.8,       # fraction of features per tree
    num_class=25,
    eval_metric='mlogloss',
    objective='multi:softmax'
)

# Training the model
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.9905
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     86275
           1       0.00      0.00      0.00        99
           2       0.00      0.00      0.00        93
           3       0.00      0.00      0.00        87
           4       0.00      0.00      0.00        96
           5       0.00      0.00      0.00        56
           6       0.00      0.00      0.00        36
           7       0.00      0.00      0.00        42
           8       0.00      0.00      0.00        39
           9       0.00      0.00      0.00        37
          10       0.00      0.00      0.00        27
          11       0.00      0.00      0.00        22
          12       0.00      0.00      0.00        20
          13       0.00      0.00      0.00        13
          14       0.00      0.00      0.00        13
          15       0.00      0.00      0.00         8
          16       0.00      0.00      0.00        11
          

c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

In [ ]:
# *********************************************************************************************************************
# Predict delay bin including 0 delay, and using class weights
# *********************************************************************************************************************

# Loading Inputs
input_features = ["wind_dir", "wind_speed", "ceiling", "visibility", "temp", "dew_pnt", "pressure", "congestion_score", "time_of_day", "time_diff"]
X = data[input_features]

# Loading Outputs (5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, inf)
y = data["delay_bin"]

# Splitting data into training and testing sets
X_train, X_test = X[:int(len(data)*0.75)], X[int(len(data)*0.75):]
y_train, y_test = y[:int(len(data)*0.75)], y[int(len(data)*0.75):]

# Building Class weights
classes = np.unique(y_train)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
weight_dict = dict(zip(classes, class_weights))

# Showing weights
print(weight_dict)

# Building the model
model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,              # fraction of samples per tree
    colsample_bytree=0.8,       # fraction of features per tree
    num_class=25,
    eval_metric='mlogloss',
    objective='multi:softmax'
)

# Mapping each training sample to its class weight
sample_weights = y_train.map(weight_dict)

# Training the model
model.fit(X_train, y_train, sample_weight=sample_weights)

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print(classification_report(y_test, y_pred))

{np.int64(0): np.float64(0.04039060960293692), np.int64(1): np.float64(26.06503740648379), np.int64(2): np.float64(33.93532467532467), np.int64(3): np.float64(42.83639344262295), np.int64(4): np.float64(46.45368888888889), np.int64(5): np.float64(67.43277419354838), np.int64(6): np.float64(69.21907284768211), np.int64(7): np.float64(91.68491228070175), np.int64(8): np.float64(101.47650485436893), np.int64(9): np.float64(129.03802469135803), np.int64(10): np.float64(132.30481012658228), np.int64(11): np.float64(160.80123076923076), np.int64(12): np.float64(186.6442857142857), np.int64(13): np.float64(204.9427450980392), np.int64(14): np.float64(282.4886486486486), np.int64(15): np.float64(337.16387096774196), np.int64(16): np.float64(261.302), np.int64(17): np.float64(418.0832), np.int64(18): np.float64(497.71809523809526), np.int64(19): np.float64(418.0832), np.int64(20): np.float64(475.0945454545455), np.int64(21): np.float64(435.50333333333333), np.int64(22): np.float64(653.255), np.

In [ ]:
# *********************************************************************************************************************
# Predict delay bin including 0 delay, and balancing 0 delay data to delay data size
# *********************************************************************************************************************

# Extract 0 delay data, and above 0 delay data
no_delay = data[data["yes_delay"] == False]
yes_delay = data[data["yes_delay"]]

# Sampling 0 delay data (THERE IS POTENTIAL DATA LEAKAGE)
no_delay_sample = no_delay.sample(n=len(yes_delay))

# Building training data sets
yes_delay_train = yes_delay[:int(len(yes_delay)*0.75)]
no_delay_train = no_delay_sample[:int(len(no_delay_sample)*0.75)]
train_data = pd.concat([yes_delay_train, no_delay_train], ignore_index=False) # combining datasets
train_data = train_data.sample(frac=1).reset_index(drop=True) # Shuffling training data

input_features = ["wind_dir", "wind_speed", "ceiling", "visibility", "temp", "dew_pnt", "pressure", "congestion_score", "time_of_day", "time_diff"]
X_train = train_data[input_features]
y_train = train_data["delay_bin"]

# Building testing data sets
yes_delay_test = yes_delay[int(len(yes_delay)*0.75):]
no_delay_test = no_delay_sample[int(len(no_delay_sample)*0.75):]
test_data = pd.concat([yes_delay_test, no_delay_test], ignore_index=False) # combining datasets
test_data = test_data.sample(frac=1).reset_index(drop=True) # Shuffling training data

input_features = ["wind_dir", "wind_speed", "ceiling", "visibility", "temp", "dew_pnt", "pressure", "congestion_score", "time_of_day", "time_diff"]
X_test = test_data[input_features]
y_test = test_data["delay_bin"]

# Building the model
model = XGBClassifier(
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,              # fraction of samples per tree
    colsample_bytree=0.8,       # fraction of features per tree
    num_class=25,
    eval_metric='mlogloss',
    objective='multi:softmax'
)

# Training the model
model.fit(X_train, y_train)

# Checking Overfitting
y_train_pred = model.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_acc:.4f}")

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

Training Accuracy: 0.6261
Accuracy: 0.4970
              precision    recall  f1-score   support

           0       0.54      0.98      0.70       839
           1       0.05      0.04      0.04       100
           2       0.14      0.06      0.09        94
           3       0.00      0.00      0.00        88
           4       0.29      0.02      0.04        97
           5       0.00      0.00      0.00        57
           6       0.00      0.00      0.00        38
           7       0.00      0.00      0.00        42
           8       0.50      0.03      0.05        40
           9       0.00      0.00      0.00        37
          10       0.00      0.00      0.00        28
          11       0.00      0.00      0.00        23
          12       0.00      0.00      0.00        20
          13       0.00      0.00      0.00        14
          14       0.00      0.00      0.00        13
          15       0.00      0.00      0.00         8
          16       0.00      0.00     

c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

In [ ]:
# *********************************************************************************************************************
# Predict delay bin including 0 delay, using class weights, and balancing 0 delay data to delay data size
# *********************************************************************************************************************

# Extract 0 delay data, and above 0 delay data
no_delay = data[data["yes_delay"] == False]
yes_delay = data[data["yes_delay"]]

# Sampling 0 delay data (THERE IS POTENTIAL DATA LEAKAGE)
no_delay_sample = no_delay.sample(n=len(yes_delay))

# Building training data sets
yes_delay_train = yes_delay[:int(len(yes_delay)*0.75)]
no_delay_train = no_delay_sample[:int(len(no_delay_sample)*0.75)]
train_data = pd.concat([yes_delay_train, no_delay_train], ignore_index=False) # combining datasets
train_data = train_data.sample(frac=1).reset_index(drop=True) # Shuffling training data

input_features = ["wind_dir", "wind_speed", "ceiling", "visibility", "temp", "dew_pnt", "pressure", "congestion_score", "time_of_day", "time_diff"]
X_train = train_data[input_features]
y_train = train_data["delay_bin"]

# Building testing data sets
yes_delay_test = yes_delay[int(len(yes_delay)*0.75):]
no_delay_test = no_delay_sample[int(len(no_delay_sample)*0.75):]
test_data = pd.concat([yes_delay_test, no_delay_test], ignore_index=False) # combining datasets
test_data = test_data.sample(frac=1).reset_index(drop=True) # Shuffling training data

input_features = ["wind_dir", "wind_speed", "ceiling", "visibility", "temp", "dew_pnt", "pressure", "congestion_score", "time_of_day", "time_diff"]
X_test = test_data[input_features]
y_test = test_data["delay_bin"]

# Building Class weights
classes = np.unique(y_train)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
weight_dict = dict(zip(classes, class_weights))

# Showing weights
print(weight_dict)

# Building the model
model = XGBClassifier(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,              # fraction of samples per tree
    colsample_bytree=0.8,       # fraction of features per tree
    num_class=25,
    eval_metric='mlogloss',
    objective='multi:softmax'
)

# Mapping each training sample to its class weight
sample_weights = y_train.map(weight_dict)

# Training the model
model.fit(X_train, y_train, sample_weight=sample_weights)

# Checking Overfitting
y_train_pred = model.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_acc:.4f}")

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
acc = accuracy_score(y_test, y_pred)
print(f"Testing Accuracy: {acc:.4f}")
print(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

{np.int64(0): np.float64(0.08), np.int64(1): np.float64(0.7733333333333333), np.int64(2): np.float64(0.58), np.int64(3): np.float64(0.696), np.int64(4): np.float64(0.9942857142857143), np.int64(5): np.float64(2.32), np.int64(6): np.float64(1.16), np.int64(7): np.float64(1.74), np.int64(8): np.float64(1.2654545454545454), np.int64(9): np.float64(2.32), np.int64(10): np.float64(3.48), np.int64(11): np.float64(2.784), np.int64(12): np.float64(13.92), np.int64(13): np.float64(13.92), np.int64(14): np.float64(3.48), np.int64(15): np.float64(13.92), np.int64(16): np.float64(3.48), np.int64(17): np.float64(6.96), np.int64(18): np.float64(4.64), np.int64(19): np.float64(3.48), np.int64(20): np.float64(6.96), np.int64(21): np.float64(13.92), np.int64(22): np.float64(13.92), np.int64(23): np.float64(4.64), np.int64(24): np.float64(0.7326315789473684)}
Training Accuracy: 0.9598
Testing Accuracy: 0.3729
              precision    recall  f1-score   support

           0       0.57      0.73      0

c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [145]:
# *********************************************************************************************************************
# Build one model to predict yes/no delay
# Which will later be used in combination with the model which predicts delay bin
# *********************************************************************************************************************

data = data[data["time_diff"]<61]

# Extract 0 delay data, and above 0 delay data
no_delay = data[data["yes_delay"] == False]
yes_delay = data[data["yes_delay"]]

# Sampling 0 delay data (THERE IS POTENTIAL DATA LEAKAGE)
no_delay_sample = no_delay.sample(n=len(yes_delay))

# Building training data sets
yes_delay_train = yes_delay[:int(len(yes_delay)*0.75)]
no_delay_train = no_delay_sample[:int(len(no_delay_sample)*0.75)]
train_data = pd.concat([yes_delay_train, no_delay_train], ignore_index=False) # combining datasets
train_data = train_data.sample(frac=1).reset_index(drop=True) # Shuffling training data

input_features = ["wind_dir", "wind_speed", "ceiling", "visibility", "temp", "dew_pnt", "pressure", "congestion_score", "time_of_day", "time_diff"]
X_train = train_data[input_features]
y_train = train_data["yes_delay"] == True

# Building testing data sets
yes_delay_test = yes_delay[int(len(yes_delay)*0.75):]
no_delay_test = no_delay_sample[int(len(no_delay_sample)*0.75):]
test_data = pd.concat([yes_delay_test, no_delay_test], ignore_index=False) # combining datasets
test_data = test_data.sample(frac=1).reset_index(drop=True) # Shuffling training data

input_features = ["wind_dir", "wind_speed", "ceiling", "visibility", "temp", "dew_pnt", "pressure", "congestion_score", "time_of_day", "time_diff"]
X_test = test_data[input_features]
y_test = test_data["yes_delay"] == True

# Building the model
model = XGBClassifier(
    n_estimators=500,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,              # fraction of samples per tree
    colsample_bytree=0.8,       # fraction of features per tree
    num_class=2,
    eval_metric='mlogloss',
    objective='multi:softmax'
)

# Training the model
model.fit(X_train, y_train)

# Checking Overfitting
y_train_pred = model.predict(X_train)
train_acc = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_acc:.4f}")

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
acc = accuracy_score(y_test, y_pred)
print(f"Testing Accuracy: {acc:.4f}")
print(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

Training Accuracy: 0.9371
Testing Accuracy: 0.6492
              precision    recall  f1-score      support
False          0.628099  0.731774  0.675985   727.000000
True           0.678748  0.566713  0.617691   727.000000
accuracy       0.649243  0.649243  0.649243     0.649243
macro avg      0.653424  0.649243  0.646838  1454.000000
weighted avg   0.653424  0.649243  0.646838  1454.000000


In [179]:
# *********************************************************************************************************************
# Build another model to predict time delay bin if yes delay
# *********************************************************************************************************************

# Extract above 0 delay data
yes_delay = data[data["yes_delay"]]

# Loading Inputs
input_features = ["wind_dir", "wind_speed", "ceiling", "visibility", "temp", "dew_pnt", "pressure", "congestion_score", "time_of_day", "time_diff"]
X = yes_delay[input_features]

# Loading Outputs
y = (yes_delay["delay_bin"]-1)//2

# Splitting data into training and testing sets
X_train, X_test = X[:int(len(yes_delay)*0.5)], X[int(len(yes_delay)*0.5):]
y_train, y_test = y[:int(len(yes_delay)*0.5)], y[int(len(yes_delay)*0.5):]

# Building the model
model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,              # fraction of samples per tree
    colsample_bytree=0.8,       # fraction of features per tree
    eval_metric='mlogloss',
    objective='multi:softmax'
)

# Training the model
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# Evaluating the model
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print(pd.DataFrame(classification_report(y_test, y_pred, output_dict=True)).transpose())

Accuracy: 0.2248
              precision    recall  f1-score      support
0              0.268608  0.631980  0.376987   394.000000
1              0.196931  0.226471  0.210670   340.000000
2              0.180000  0.136364  0.155172   198.000000
3              0.107143  0.018868  0.032086   159.000000
4              0.052632  0.015625  0.024096   128.000000
5              0.047619  0.013514  0.021053    74.000000
6              0.000000  0.000000  0.000000    61.000000
7              0.000000  0.000000  0.000000    46.000000
8              0.000000  0.000000  0.000000    39.000000
9              0.000000  0.000000  0.000000    31.000000
10             0.000000  0.000000  0.000000    27.000000
11             0.163636  0.100000  0.124138   180.000000
accuracy       0.224806  0.224806  0.224806     0.224806
macro avg      0.084714  0.095235  0.078684  1677.000000
weighted avg   0.158127  0.224806  0.168738  1677.000000


c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\abrah\Desktop\Projects\wi_environment\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 